In [1]:
import time, datetime
import codecs
import json
from TwitterAPI import TwitterAPI
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

#TwitterAPI Settings
api = TwitterAPI('OOheRS0hK5FQaj6iL75XTILke', 'aDeZ9i3dZ18KjtDiEcNf683hwjKC4MLyAR3v1f1FqHiW3QdNEz', '831073699411275776-7HDtuctKY7orzuEIRPhEkChTgo2JJ5n', 'txslgzywJrEMmy6zi2e8AdpikcqsWo6k9hueSJMibSeZx')

#Open webdriver Chrome
#Use location to your chromedriver.exe
browser = webdriver.Chrome("chromedriver.exe")

#search url
#a
# browser.get("https://twitter.com/search?f=tweets&vertical=default&q=diare%20OR%20imodium%20OR%20diapet%20OR%20%22sakit%20perut%22%20OR%20mules%20since%3A2018-12-22%20until%3A2018-12-23&l=id&src=spxr")
#b
browser.get("https://twitter.com/search?f=tweets&vertical=default&q=mencret%20OR%20menceret%20OR%20salmonella%20OR%20%22e%20coli%22%20OR%20oralit%20since%3A2018-10-01%20until%3A2018-12-31&l=id&src=spxr")

lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
match = False
while (match==False):
    lastCount = lenOfPage
    time.sleep(3)
    lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
    if lastCount == lenOfPage:
        match=False

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=73.0.3683.103)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)


In [5]:
from TwitterAPI import TwitterAPI

api = TwitterAPI('OOheRS0hK5FQaj6iL75XTILke', 'aDeZ9i3dZ18KjtDiEcNf683hwjKC4MLyAR3v1f1FqHiW3QdNEz', '831073699411275776-7HDtuctKY7orzuEIRPhEkChTgo2JJ5n', 'txslgzywJrEMmy6zi2e8AdpikcqsWo6k9hueSJMibSeZx')
r = api.request('statuses/show/:%d' % 947775136970797056)
for item in r.get_iterator():
    #Place ID
    placeID = item['place']['id']
    req = api.request('geo/id/:%s' % placeID)
    json_var = req.json()
json_var

{'id': '2eb44917f4d6dddc',
 'url': 'https://api.twitter.com/1.1/geo/id/2eb44917f4d6dddc.json',
 'place_type': 'city',
 'name': 'Cirebon Barat',
 'full_name': 'Cirebon Barat, Indonesia',
 'country_code': 'ID',
 'country': 'Indonesia',
 'contained_within': [{'id': '388445308dc7f4b5',
   'url': 'https://api.twitter.com/1.1/geo/id/388445308dc7f4b5.json',
   'place_type': 'admin',
   'name': 'Cirebon',
   'full_name': 'Cirebon, Jawa Barat',
   'country_code': 'ID',
   'country': 'Indonesia',
   'centroid': [108.45057481324469, -6.756577],
   'bounding_box': {'type': 'Polygon',
    'coordinates': [[[108.321145, -6.99836],
      [108.321145, -6.514794],
      [108.8338797, -6.514794],
      [108.8338797, -6.99836],
      [108.321145, -6.99836]]]},
   'attributes': {}}],
 'geometry': None,
 'polylines': [],
 'centroid': [108.53915156648996, -6.7191105],
 'bounding_box': {'type': 'Polygon',
  'coordinates': [[[108.5115572, -6.751051],
    [108.5115572, -6.68717],
    [108.562858, -6.68717],
   

In [4]:
#Open session to write
outputFile = codecs.open("dia sto.csv", "w+", "utf-8")
outputFile.write('username,date,time,text,tweetID,placeID,daerah,lokasi,longitude,latitude\n')

#tweet_contents 
tweet_contents = browser.find_elements_by_class_name("js-stream-tweet")

index = 0

for tweet in tweet_contents[index:]:
    #Tweet text
    text = tweet.find_element_by_css_selector("div.js-tweet-text-container > p").text.replace('\n',' ').replace(',', ' ')
    #Username
    username = '@'+tweet.find_element_by_css_selector("div.stream-item-header > a > span.username > b").text
    #Date of tweet
    dateInt = int(tweet.find_element_by_css_selector("div.stream-item-header > small > a > span").get_attribute('data-time'))
    dateFormat = datetime.datetime.fromtimestamp(dateInt)
    #Tweet ID
    tweetID = int(tweet.get_attribute("data-tweet-id"))
    
    r = api.request('statuses/show/:%d' % tweetID)
    if r.status_code != 200:
        print('false status')
        print(r.status_code)
        print(username)
        print(tweetID)
    else:
        for item in r.get_iterator():
            #Place ID
            if item['place'] != None:
                placeID = item['place']['id']
                r = api.request('geo/id/:%s' % placeID)

                if r.status_code != 200:
                    print('false place')
                    print(r.status_code)
                    print(placeID)
                else:
                    json_var = r.json()
                    #name
                    daerah = json_var['name']
                    #fullname
                    lokasi = json_var['full_name'].replace(',', '')
                    #longitude
                    longitude = json_var['centroid'][0]
                    #latitude
                    latitude = json_var['centroid'][1]
            else:
                placeID = ''
                daerah = ''
                lokasi = ''
                longitude = ''
                latitude = ''
        #test
        print(index)
        print('username: ' + username)
        print('date: ' + dateFormat.strftime("%Y-%m-%d"))
        print('time: ' + dateFormat.strftime("%H:%M"))
        print('text: ' + text)
        print('tweet id: %s ' % tweetID)
        print('place id: %s ' % placeID)
        print('daerah: ' + daerah)
        print('lokasi: ' + lokasi)
        print('longitude: %s ' % longitude)
        print('latitude: %s ' % latitude)
        print()
        index = index + 1
        outputFile.write('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s\n' % (username, dateFormat.strftime("%Y-%m-%d"), dateFormat.strftime("%H:%M"), text, tweetID, placeID, daerah, lokasi, longitude, latitude))

0
username: @MulyadiMade
date: 2018-12-31
time: 06:58
text: Mencret di isu Agama ? @AkunTofa melon coba peruntungan di PKI...sekarang otaknya di penuhi PKI  hanya seorang PKI sejatilah yg otaknya penuh  tiap hari mikirin dan mencari PKI...
tweet id: 1079527238414913536 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

1
username: @kattenkendi
date: 2018-12-31
time: 06:43
text: baru sadar tadi gue mencret mencret
tweet id: 1079523315775987712 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

2
username: @wiwitnurafifah
date: 2018-12-31
time: 06:35
text: Sianida aja udah bikin mati  e.coli udh gak berguna
tweet id: 1079521456843677696 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

3
username: @andanidind
date: 2018-12-31
time: 04:17
text: Dikira mencret seumur hidup apa
tweet id: 1079486628375158784 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

4
username: @gelebugelebug
date: 2018-12-31
time: 03:57
text: lelaki kardus lelaki kampret lelaki kencrot lela

37
username: @alanaideen
date: 2018-12-30
time: 21:43
text: Auto mencret ya ga si 1 aja udh lancar bet kek tol kosong melempeng
tweet id: 1079387605911793664 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

38
username: @pdrwjy
date: 2018-12-30
time: 21:31
text: M4 MALAM MINGGU MENCRET MENCRET
tweet id: 1079384504073019392 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

39
username: @revanheryan08
date: 2018-12-30
time: 21:16
text: gw sangat sadar karena lu gak pernah berargumen nyata dan menggunakan logika...fanatisme boleh selama prinsip lu juga gak berubah...disaat fanatisme lu dimencretin sama logika ya otak lu jadi ngawur  disitulah idealis lu jadi terukur....berak mencret atau berak dicelana  behahaha
tweet id: 1079380758072979456 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

40
username: @raspbeerryyy
date: 2018-12-30
time: 21:16
text: kalo sakit diare namanya mencret dong
tweet id: 1079380710568321024 
place id:  
daerah: 
lokasi: 
longitude:  
latit

72
username: @dxjeka
date: 2018-12-30
time: 16:58
text: Jogan lelaki kancut lelaki mencret
tweet id: 1079315778636152833 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

73
username: @elanda_rahmat
date: 2018-12-30
time: 16:58
text: Mencret njiiir
tweet id: 1079315752006701057 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

74
username: @teguhprasaja
date: 2018-12-30
time: 16:51
text: Hah.. Bisnis men.. Mencret..
tweet id: 1079314036838125569 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

75
username: @urcegan
date: 2018-12-30
time: 16:50
text: Humor aku mencret ni tulung
tweet id: 1079313654661533697 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

76
username: @Jenniebimy
date: 2018-12-30
time: 16:48
text: Ihh mencret ewhh 
tweet id: 1079313324985053184 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

77
username: @armawaras
date: 2018-12-30
time: 16:48
text: Dari Gombong sampe Kebakkramat  Banyak pedagang pasar jual bendera.. Ngomong-ngomon

106
username: @oktaviaprsr_
date: 2018-12-30
time: 13:05
text: Suami mandiri  alhamdulillah tidak jadi lelaki mencret dan kampret  wkwkwkkwk 
tweet id: 1079257046187335680 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

107
username: @aguspakpahan
date: 2018-12-30
time: 13:00
text: Penyakit tropika akibat Coli dan Salmonella 
tweet id: 1079255826479558657 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

108
username: @masbhoed
date: 2018-12-30
time: 12:34
text: Mending gw denger suara mencret anak gw dah
tweet id: 1079249386431868928 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

109
username: @lovelyzkei
date: 2018-12-30
time: 12:28
text: Epriciet kebaca mencret   
tweet id: 1079247849215291392 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

110
username: @puturadion
date: 2018-12-30
time: 12:18
text: Kalo lo mencret sekarang makanya jangan berak sebelumnya  gini lah kalo bawa2 agama ke politik terus di jabani jadi ngehek kan?
tweet id: 107924535911

139
username: @Anak_Perantau_
date: 2018-12-30
time: 01:48
text: Wowo mencret....sepanjang acara mulai....
tweet id: 1079086665396740096 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

140
username: @FOODFESS
date: 2018-12-30
time: 01:45
text: ffess Bagi yang punya kesempatan untuk ke Jepang  monggo dicicip: CHICKEN TATAKI  yaitu sashimi ayam a.k.a. ayam mentah. Tenang  ayam yang digunakan bersih kok  enggak bikin salmonella! 
tweet id: 1079085932190429184 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

141
username: @daeysic
date: 2018-12-30
time: 01:45
text: SEBENERNYA GUE GA MAU MIKIRIN KEK GINIAN  TP SERING KEPIKIRAN SENDIRI WKWKWKWKWK  atau ga pas mereka lagi mencret gitu ya :((
tweet id: 1079085930399420417 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

142
username: @afrizal_umami
date: 2018-12-30
time: 01:31
text: Laca Ozil Mikhi nndi nang? Mencret po?
tweet id: 1079082533604646912 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

143
username

...

7989
username: @Gyuyeomine
date: 2018-10-03
time: 10:53
text: Udah bolak balik ke wc brapa kli nder? Oralit aja
tweet id: 1047333842825035776 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7990
username: @BKIPMAmbon
date: 2018-10-03
time: 10:48
text: 3/10 Persiapan Penambahan Ruang Lingkup Laboratorium Berupa Pengujian Organoleptik Menggunakan Metode SNI Serta Pengujian Salmonella sp dan Escherichia coli dengan Menggunakan Metode ISO #oneHealthKIPM
tweet id: 1047332483883139072 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7991
username: @Kcbong4
date: 2018-10-03
time: 10:28
text: Artisnya Randa sarikampret.. Sutradaranya Pageli nyong... Skenario Mr Mencret 
tweet id: 1047327450839957504 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7992
username: @Fabiano07626412
date: 2018-10-03
time: 10:19
text: Koe ki ngomong opo mencret.....
tweet id: 1047325127640154112 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

7993
username: @pret_dutt
date: 2018-10

8024
username: @acchyeri
date: 2018-10-02
time: 20:59
text: Mamam mencret aku yah?:(
tweet id: 1047123997853270016 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8025
username: @acchyeri
date: 2018-10-02
time: 20:53
text: Bntr ya tungguin aku mencret nanti kamu boleh jilat
tweet id: 1047122299881582592 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8026
username: @ethereal_blosso
date: 2018-10-02
time: 20:53
text: Mereka alhamdulillah sehat. Jarang sakit. Paling cuma pilek ringan aja. Matanya juga sehat banget. Ga pernah mencret juga. Soalnya entah kenapa  mereka pinter bedain mana makanan enak sama engga /sama mahal apa murah . Dasar kucing2 matre wkwk
tweet id: 1047122290633203713 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8027
username: @2gudangbegundal
date: 2018-10-02
time: 20:34
text: Kalo masalah gak lapor....itu kan karena gk percaya REZIM sekarang..... Yg aneh....kok ada tai lalat mencret di mukanya
tweet id: 1047117531725451264 
place id:  
d

8059
username: @brodo_c
date: 2018-10-02
time: 14:53
text: hasil pemikiran otak mencret ..
tweet id: 1047031896499216384 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8060
username: @Otto73351888
date: 2018-10-02
time: 14:31
text: @LawanPoLitikJW ada org penjilat dan buka setan anak mencret gempa semua salah jokowi demi tenarnya dan setiap hari bahan racuni org.. Muka sj sdh percis spt inang2 tkg jengkol ..wioiiii ferdinand jgn jd sipelebegu kau
tweet id: 1047026355244806146 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8061
username: @ntashadc
date: 2018-10-02
time: 14:07
text: YAELAH AKU LEVEL 8 PERNAH AHAHAHAH  abistu mencret :(((
tweet id: 1047020335508393985 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8062
username: @takiif
date: 2018-10-02
time: 13:25
text: seng Bender sampe mencret Gus
tweet id: 1047009655371829249 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8063
username: @markeinct
date: 2018-10-02
time: 13:18
text: Lelaki mencre

8091
username: @alvinmalana
date: 2018-10-01
time: 23:23
text: 17. Obat-obatan dengan fokus pada 3 hal: pengobat luka luar (perban  antiseptik  plester)  obat diare (oralit  norit) serta pengusir nyamuk di ruang terbuka (risiko malaria/DB).
tweet id: 1046797823511953408 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8092
username: @baasel__
date: 2018-10-01
time: 23:05
text: Wakaka lgsg mencret cak
tweet id: 1046793107981959168 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8093
username: @baasel__
date: 2018-10-01
time: 22:53
text: Uda tau ga tawar sama kopi masi beli juga  kmren perut g enak skrg mencret. Ya sama kayak uda tau ucok fana tp masi naksir juga  
tweet id: 1046790144873791490 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8094
username: @jjaehyont
date: 2018-10-01
time: 22:49
text: Apaan orang tadi gua mencret kok.
tweet id: 1046789150974722049 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8095
username: @Danadan15596181
date: 2018

8125
username: @alemdawo
date: 2018-10-01
time: 13:45
text: Oralit
tweet id: 1046652222321319936 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8126
username: @lovessyoonoh
date: 2018-10-01
time: 13:42
text: oralit
tweet id: 1046651552788672512 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8127
username: @kagapunya
date: 2018-10-01
time: 13:35
text: Pernahnya grgr makan kebanyakan  terus muntah. Sampe sekarang gamau muntah lagi. Mendingan mencret mencret dah gue dibanding muntah:'(
tweet id: 1046649844117295105 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8128
username: @aprodet
date: 2018-10-01
time: 13:24
text: Itu ayamnya lagi mencret ya bisa moncrot gitu wkwkwk
tweet id: 1046646916652490752 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

8129
username: @wonknaim
date: 2018-10-01
time: 12:44
text: Salah kamarr bung..nimbrung ajaa...pokoknnya salawi... Lo berak mencret pun salah jokowi..
tweet id: 1046636931017203713 
place id:  
daerah: 
lok

In [3]:
outputFile = codecs.open("dia des a4 sto.csv", "w+", "utf-8")
outputFile.write('username,date,time,text,tweetID,placeID,daerah,lokasi,longitude,latitude\n')

#tweet_contents 
tweet_contents = browser.find_elements_by_class_name("js-stream-tweet")

index = 900

for tweet in tweet_contents[index:]:
    #Tweet text
    text = tweet.find_element_by_css_selector("div.js-tweet-text-container > p").text.replace('\n',' ').replace(',', ' ')
    #Username
    username = '@'+tweet.find_element_by_css_selector("div.stream-item-header > a > span.username > b").text
    #Date of tweet
    dateInt = int(tweet.find_element_by_css_selector("div.stream-item-header > small > a > span").get_attribute('data-time'))
    dateFormat = datetime.datetime.fromtimestamp(dateInt)
    #Tweet ID
    tweetID = int(tweet.get_attribute("data-tweet-id"))
    
    r = api.request('statuses/show/:%d' % tweetID)
    if r.status_code != 200:
        print('false status')
        print(r.status_code)
        print(username)
        print(tweetID)
    else:
        for item in r.get_iterator():
            #Place ID
            if item['place'] != None:
                placeID = item['place']['id']
                r = api.request('geo/id/:%s' % placeID)

                if r.status_code != 200:
                    print('false place')
                    print(r.status_code)
                    print(placeID)
                else:
                    json_var = r.json()
                    #name
                    daerah = json_var['name']
                    #fullname
                    lokasi = json_var['full_name'].replace(',', '')
                    #longitude
                    longitude = json_var['centroid'][0]
                    #latitude
                    latitude = json_var['centroid'][1]
            else:
                placeID = ''
                daerah = ''
                lokasi = ''
                longitude = ''
                latitude = ''
        #test
        print(index)
        print('username: ' + username)
        print('date: ' + dateFormat.strftime("%Y-%m-%d"))
        print('time: ' + dateFormat.strftime("%H:%M"))
        print('text: ' + text)
        print('tweet id: %s ' % tweetID)
        print('place id: %s ' % placeID)
        print('daerah: ' + daerah)
        print('lokasi: ' + lokasi)
        print('longitude: %s ' % longitude)
        print('latitude: %s ' % latitude)
        print()
        index = index + 1
        outputFile.write('%s, %s, %s, %s, %s, %s, %s, %s, %s, %s\n' % (username, dateFormat.strftime("%Y-%m-%d"), dateFormat.strftime("%H:%M"), text, tweetID, placeID, daerah, lokasi, longitude, latitude))

900
username: @jati04
date: 2018-12-22
time: 07:21
text: Beli saham ..sakit perut minum obat oskadon ...terlalu pinter x ya
tweet id: 1076271523210514435 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

901
username: @sayap_pedas
date: 2018-12-22
time: 07:21
text: Sedang menunggu mules datang  baru berangkat lari
tweet id: 1076271403211452416 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

902
username: @eucatastrope
date: 2018-12-22
time: 07:18
text: Elah gw mules
tweet id: 1076270666771357696 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

903
username: @bluecherrynim
date: 2018-12-22
time: 07:16
text: Engga ko tenang aja Kalo sakit perut paling cuma diare/?
tweet id: 1076270286234779649 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

904
username: @azizyordafasha
date: 2018-12-22
time: 07:16
text: Dan saya pun dibuat mules baca komen2 nya. Hahahahahaha 
tweet id: 1076270177589723136 
place id:  
daerah: 
lokasi: 
longitude:  
latitude:  

905
usern

In [6]:
test = []
test.append(['a'])

In [9]:
test[0][0]

'a'